# Where to open another chain of your restaurant in Toronto
## Introduction/Business Problem
When you are a successful entrepreneur in a city, say in this case Toronto, owning a well running restaurant, you could naturally think about opening another chain store. Through clustering, the entrepreneur can find similar spots that might replicate the success of the existing business.
## Data Section
I will continue working on the data presented in previous weeks:  
1. Borough and neighborhood data scraping from: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. This gives a basic idea of different neighborhood
2. Latitude and longitude provided in the file “Geospatial_Coordinates.csv”. This tells us the latitude and longitude information of the neighborhood
3. Using Foursquare to acquire business information, so that neighborhood with similar business type could be clustered together. 